In [ ]:
import os

import cmcrameri.cm as cmc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats



In [2]:
def myround(x, base=5):
    return base * round(x/base)

In [3]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined.csv')

In [4]:
def get_stf(scardec_name, wanted_type = 'fctopt'):
    db = combined[combined['scardec_name']==scardec_name]

    time = []
    momentrate = []

    event = os.listdir(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}')
    starts = [n for n, l in enumerate(event) if l.startswith(wanted_type)]
    with open(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec/{scardec_name}/{event[starts[0]]}') as f:
        lines = f.read().splitlines()

    lines = lines[2:]
    for line in lines:
        split = line.split(' ')
        split = [s for s in split if s not in ['', ' ', '\n']]
        time.append(float(split[0]))
        momentrate.append(float(split[1]))

    momentrate = np.array(momentrate)
    return momentrate, time, db

In [5]:
# looks for time value of root
def f3(end_time, total_moment, time_opt, momentrate_opt, start, points_before_zero, proportion = 0.1):
    dx = time_opt[1]-time_opt[0]
    end_window = (end_time/dx)+points_before_zero
    end = int(np.floor(end_window))
    short = scipy.integrate.simpson(momentrate_opt[start:end], dx = dx)
    return short-(total_moment*proportion)

In [6]:
def plot_scatter_figures(proportion, root_times, moments, durations):
    root_times = np.array(root_times)
    moments = np.array(moments)
    durations = np.array(durations)

    plt.scatter(root_times, durations, c = np.log10(moments), cmap = cmc.batlow, alpha = 0.5)
    plt.ylabel('Duration (s)')
    plt.xlabel(f'time to release {proportion*100}% of moment (s)')
    plt.colorbar(label = 'log10(moment)')
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/time_for_{proportion*100}_percent_moment_against_duration.png')
    plt.close()

    plt.scatter(root_times, np.log10(moments), c = durations, cmap = cmc.batlow, alpha = 0.5)
    plt.ylabel('log10(moment)')
    plt.xlabel(f'time to release {proportion*100}% of moment (s)')
    plt.colorbar(label = 'Duration (s)')
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/time_for_{proportion*100}_percent_moment_against_moment.png')
    plt.close()

    plt.scatter(root_times/durations, np.log10(moments), c = durations, cmap = cmc.batlow, alpha = 0.5)
    plt.ylabel('log10(moment)')
    plt.xlabel(f'proportion of duration to release {proportion*100}% of moment')
    plt.colorbar()
    plt.xlim(0, 1)
    plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/fraction_of_duration_for_{proportion*100}_percent_moment_against_moment.png')
    plt.close()

In [7]:
def plot_hist_figures(proportion, root_times, durations):
    root_times = np.array(root_times)
    durations = np.array(durations)

    plt.hist(root_times/durations, bins = 100)

    plt.ylabel('Frequency')
    plt.xlabel(f'Proportion of duration to release {proportion*100}% of moment')
    plt.xlim(0, 1)
    plt.show()
    #plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/histogram_fraction_of_duration_for_{proportion*100}_percent_moment.png')
    #plt.close()

In [ ]:
proportions_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

all_durations = []
all_root_times = []
all_moments = []
all_relative_root_times = []

df = pd.DataFrame(columns = ['proportion', 'skew', 'skew statistic', 'skew pvalue',  'kurtosis', 'kurtosis statistic', 'kurtosis pvalue'])

for proportion in proportions_list:
    print(proportion)
    durations = []
    root_times = []
    relative_root_times = []

    diff = []
    moments = []

    for scardec_name in os.listdir('/home/earthquakes1/homes/Rebecca/phd/stf/data/scardec'):
        #print(scardec_name)
        momentrate_opt, time_opt, db = get_stf(scardec_name)

        not_zero = np.where(momentrate_opt > 0)[0]

        dx = time_opt[1]-time_opt[0]

        start = min(not_zero)
        end = max(not_zero)
        points_before_zero = abs(min(time_opt)/dx)

        duration = time_opt[end] - time_opt[start]
        durations.append(duration)

        start_time = time_opt[start]
        end_time = time_opt[end]

        total_moment = scipy.integrate.simpson(momentrate_opt[start:end], dx = time_opt[1]-time_opt[0])
        moments.append(total_moment)
        root, r = scipy.optimize.bisect(f3,
                                        start_time+dx,
                                        end_time,
                                        rtol = 1e-6,
                                        full_output = True,
                                        args = (total_moment,
                                                time_opt,
                                                momentrate_opt,
                                                start,
                                                points_before_zero,
                                                proportion,))
        root_idx = np.floor(root/dx)
        root_time = root_idx*dx
        root_times.append(root_time)
        relative_root_times.append(root_time-start_time)

        if root_time-start_time > duration:
            print('root time greater than duration, proportion:', proportion)
            print(scardec_name)

    root_times = np.array(root_times)
    durations = np.array(durations)
    moments = np.log10(np.array(moments))
    relative_root_times = np.array(relative_root_times)

    rel_root_times = relative_root_times/durations

    skew = stats.skew(rel_root_times)
    kurtosis = stats.kurtosis(rel_root_times)
    kurtosis_test = stats.kurtosistest(rel_root_times)
    skew_test = stats.skewtest(rel_root_times)

    new_data = {'proportion': [proportion], 'skew': [skew], 'skew statistic': [skew_test[0]], 'skew pvalue': [skew_test[1]], 'kurtosis': [kurtosis], 'kurtosis statistic': [kurtosis_test[0]], 'kurtosis pvalue': [kurtosis_test[1]]}
    df = pd.concat([df, pd.DataFrame(new_data)])


    #plt.savefig(f'/home/earthquakes1/homes/Rebecca/phd/stf/figures/moment_intervals/stacked_histogram_fraction_of_duration_for_{proportion*100}_percent_moment.png')

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9


Skew: For normally distributed data, the skewness should be about zero. For unimodal continuous distributions, a skewness value greater than zero means that there is more weight in the right tail of the distribution. 

Skew statistic: The computed z-score for this test.

Skew p-value: The test is performed by comparing the observed value of the statistic against the null distribution: the distribution of statistic values derived under the null hypothesis that the weights were drawn from a normal distribution.




In [29]:
df

,proportion,skew,skew statistic,skew pvalue,kurtosis,kurtosis statistic,kurtosis pvalue
0,0.1,0.739420,17.369639,1.401055e-67,1.552352,11.962593,5.579212e-33
0,0.2,0.490477,12.162870,4.900637e-34,0.717904,7.068803,1.562755e-12
0,0.3,0.305031,7.803642,6.014554e-15,0.270461,3.184645,1.449316e-03
0,0.4,0.174825,4.538177,5.674263e-06,0.141811,1.792034,7.312748e-02
0,0.5,0.064235,1.678225,9.330313e-02,0.134856,1.712186,8.686242e-02
0,0.6,-0.047512,-1.241883,2.142798e-01,0.271789,3.198249,1.382647e-03
0,0.7,-0.148663,-3.866972,1.101950e-04,0.518466,5.492172,3.970213e-08
0,0.8,-0.207739,-5.376389,7.599469e-08,1.061309,9.356345,8.254281e-21
0,0.9,-0.313581,-8.012840,1.120891e-15,4.036254,19.568891,2.848094e-85


In [30]:
print(df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.3f}".format,
))

\begin{tabular}{rrrrrrr}
\toprule
proportion & skew & skew statistic & skew pvalue & kurtosis & kurtosis statistic & kurtosis pvalue \\
\midrule
0.100 & 0.739 & 17.370 & 0.000 & 1.552 & 11.963 & 0.000 \\
0.200 & 0.490 & 12.163 & 0.000 & 0.718 & 7.069 & 0.000 \\
0.300 & 0.305 & 7.804 & 0.000 & 0.270 & 3.185 & 0.001 \\
0.400 & 0.175 & 4.538 & 0.000 & 0.142 & 1.792 & 0.073 \\
0.500 & 0.064 & 1.678 & 0.093 & 0.135 & 1.712 & 0.087 \\
0.600 & -0.048 & -1.242 & 0.214 & 0.272 & 3.198 & 0.001 \\
0.700 & -0.149 & -3.867 & 0.000 & 0.518 & 5.492 & 0.000 \\
0.800 & -0.208 & -5.376 & 0.000 & 1.061 & 9.356 & 0.000 \\
0.900 & -0.314 & -8.013 & 0.000 & 4.036 & 19.569 & 0.000 \\
\bottomrule
\end{tabular}



In [11]:
stats.skewtest(rel_root_times)

SkewtestResult(statistic=17.369638641121778, pvalue=1.4010547739410029e-67)